In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

### Testing keras tuner on IRIS dataset

In [2]:
from sklearn.datasets import load_iris
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

iris_ds = load_iris()
X = iris_ds.data
# convert integers to dummy variables (i.e. one hot encoded)
y_one_hot_encoded = to_categorical(iris_ds.target)
y_one_hot_encoded[:4]


array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot_encoded, test_size=0.33, random_state=42)



Usage: the basics
Here's how to perform hyperparameter tuning for a single-layer dense neural network using: 
  * random search
  * bayesian optimization
  * hyperband

First, we define a model-building function. It takes an argument hp from which you can sample hyperparameters, such as hp.Int('units', min_value=32, max_value=512, step=32) (an integer from a certain range).

This function returns a compiled model.

In [4]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(input_dim=X.shape[1],
                            units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                            
                           activation='relu'))
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model


Next, instantiate a tuner. You should specify the model-building function, the name of the objective to optimize (whether to minimize or maximize is automatically inferred for built-in metrics), the total number of trials (max_trials) to test, and the number of models that should be built and fit for each trial (executions_per_trial).

Available tuners are RandomSearch and Hyperband (y bayesian?)

Note: the purpose of having multiple executions per trial is to reduce results variance and therefore be able to more accurately assess the performance of a model. If you want to get results faster, you could set executions_per_trial=1 (single round of training for each model configuration).

In [5]:
import os

random_search_tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    #directory=r'./KERAS_TUNER_CHECKS',
    #directory=os.path.normpath('.\keras_tuner_checks_logs'),
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_random_search_tuner_test')


INFO:tensorflow:Reloading Oracle from existing project C:\keras_tuning\iris_data_keras_random_search_tuner_test\oracle.json
INFO:tensorflow:Reloading Tuner from C:\keras_tuning\iris_data_keras_random_search_tuner_test\tuner0.json


In [6]:
from kerastuner import BayesianOptimization

bayesian_opt_tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    #directory=r'./KERAS_TUNER_CHECKS',
    #directory=os.path.normpath('.\keras_tuner_checks_logs'),
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_bayesian_opt_tuner_test')


INFO:tensorflow:Reloading Oracle from existing project C:\keras_tuning\iris_data_keras_bayesian_opt_tuner_test\oracle.json
INFO:tensorflow:Reloading Tuner from C:\keras_tuning\iris_data_keras_bayesian_opt_tuner_test\tuner0.json


summary of the search space:
Start the search for the best hyperparameter configuration. The call to search has the same signature as model.fit().
Here's what happens in search: models are built iteratively by calling the model-building function, which populates the hyperparameter space (search space) tracked by the hp object. The tuner progressively explores the space, recording metrics for each configuration.

In [7]:
random_search_tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test)
             #validation_split=0.2,verbose=1)
             )


INFO:tensorflow:Oracle triggered exit


In [8]:
random_search_tuner.results_summary()


In [9]:
bayesian_opt_tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test)
             #validation_split=0.2,verbose=1)
             )

INFO:tensorflow:Oracle triggered exit


In [10]:
bayesian_opt_tuner.results_summary()


## Best models achieved with the random search and and bayesian hyperparametrization

In [11]:
rand_searched_models = random_search_tuner.get_best_models(num_models=-1)
bayes_optimized_models = bayesian_opt_tuner.get_best_models(num_models=-1)

#tuner.get_best_hyperparameters()
print('number of random searched models: {}'.format(len(rand_searched_models)))
print('number of bayesian optimized models: {}'.format(len(bayes_optimized_models)))


number of random searched models: 4
number of bayesian optimized models: 4


## Get model weights:

In [12]:
random_searched_model_best_model = random_search_tuner.get_best_models(num_models=1)
bayes_opt_model_best_model = bayesian_opt_tuner.get_best_models(num_models=1)



 ### Evaluation score:
print('single prediction on a test instance: {}'.format(models[0].predict(X_test[-1].reshape(1, -1))))

In [13]:
print('single prediction of the random_searched_model_best_model on a test instance: {}'.format(random_searched_model_best_model[0].predict(X_test[-1].reshape(1, -1))))
print('single prediction of the bayes_opt_model_best_model on a test instance: {}'.format(bayes_opt_model_best_model[0].predict(X_test[-1].reshape(1, -1))))


single prediction of the random_searched_model_best_model on a test instance: [[0.02353263 0.42498332 0.55148405]]
single prediction of the bayes_opt_model_best_model on a test instance: [[0.00119137 0.20332347 0.79548514]]


## Predictions on the test set:

In [14]:
test_set_random_searched_best_model_predictions = random_searched_model_best_model[0].predict(X_test)
test_set_bayes_opt_best_model_predictions = bayes_opt_model_best_model[0].predict(X_test)


### Esperamos obtener un score = 0.90 para el best_model_0 según el 'Results summary'

In [15]:
import tensorflow

categ_acc_rand_search_test_set = tensorflow.keras.metrics.CategoricalAccuracy()
#best_model_0_eval_set_acc = tensorflow.keras.metrics.CategoricalAccuracy(y_test.reshape(-1,3), test_set_predictions.reshape(-1,3))
_ = categ_acc_rand_search_test_set.update_state(y_test, test_set_random_searched_best_model_predictions)
print('random_searched_model_best_model_eval_set_acc: {}'.format(categ_acc_rand_search_test_set.result().numpy()))

categ_acc_bayes_opt_test_set = tensorflow.keras.metrics.CategoricalAccuracy()
_ = categ_acc_bayes_opt_test_set.update_state(y_test, test_set_bayes_opt_best_model_predictions)
print('bayes_opt_model_best_model_eval_set_acc: {}'.format(categ_acc_bayes_opt_test_set.result().numpy()))



random_searched_model_best_model_eval_set_acc: 0.8799999952316284
bayes_opt_model_best_model_eval_set_acc: 0.9800000190734863


## Re-run model trainings and hyperparam.

In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

### Testing keras tuner on IRIS dataset

In [2]:
from sklearn.datasets import load_iris
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

iris_ds = load_iris()
X = iris_ds.data
# convert integers to dummy variables (i.e. one hot encoded)
y_one_hot_encoded = to_categorical(iris_ds.target)
y_one_hot_encoded[:4]


array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot_encoded, test_size=0.33, random_state=42)



Usage: the basics
Here's how to perform hyperparameter tuning for a single-layer dense neural network using: 
  * random search
  * bayesian optimization
  * hyperband

First, we define a model-building function. It takes an argument hp from which you can sample hyperparameters, such as hp.Int('units', min_value=32, max_value=512, step=32) (an integer from a certain range).

This function returns a compiled model.

In [4]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(input_dim=X.shape[1],
                            units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                            
                           activation='relu'))
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model


Next, instantiate a tuner. You should specify the model-building function, the name of the objective to optimize (whether to minimize or maximize is automatically inferred for built-in metrics), the total number of trials (max_trials) to test, and the number of models that should be built and fit for each trial (executions_per_trial).

Available tuners are RandomSearch and Hyperband (y bayesian?)

Note: the purpose of having multiple executions per trial is to reduce results variance and therefore be able to more accurately assess the performance of a model. If you want to get results faster, you could set executions_per_trial=1 (single round of training for each model configuration).

In [5]:
import os

random_search_tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    #directory=r'./KERAS_TUNER_CHECKS',
    #directory=os.path.normpath('.\keras_tuner_checks_logs'),
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_random_search_tuner_test',
    overwrite=True)


In [6]:
from kerastuner import BayesianOptimization

bayesian_opt_tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    #directory=r'./KERAS_TUNER_CHECKS',
    #directory=os.path.normpath('.\keras_tuner_checks_logs'),
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_bayesian_opt_tuner_test',
    overwrite=True)


summary of the search space:
Start the search for the best hyperparameter configuration. The call to search has the same signature as model.fit().
Here's what happens in search: models are built iteratively by calling the model-building function, which populates the hyperparameter space (search space) tracked by the hp object. The tuner progressively explores the space, recording metrics for each configuration.

In [7]:
random_search_tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test)
             #validation_split=0.2,verbose=1)
             )


Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 1s - loss: 1.1651 - accuracy: 0.0000e+100/100 [==============================] - 1s 8ms/sample - loss: 1.1012 - accuracy: 0.3200 - val_loss: 1.0219 - val_accuracy: 0.3200
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.9637 - accuracy: 0.40100/100 [==============================] - 0s 190us/sample - loss: 0.9973 - accuracy: 0.3400 - val_loss: 0.9304 - val_accuracy: 0.3200
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.9649 - accuracy: 0.25100/100 [==============================] - 0s 977us/sample - loss: 0.8970 - accuracy: 0.3600 - val_loss: 0.8110 - val_accuracy: 0.9600
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.8181 - accuracy: 0.87100/100 [==============================] - 0s 189us/sample - loss: 0.7958 - accuracy: 0.9200 - val_loss: 0.7153 - val_accuracy: 0.7200
Epoch 5/5
 32/100 [========>.................

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.1095 - accuracy: 0.31100/100 [==============================] - 0s 4ms/sample - loss: 1.0664 - accuracy: 0.3400 - val_loss: 1.0015 - val_accuracy: 0.4800
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.9614 - accuracy: 0.59100/100 [==============================] - 0s 938us/sample - loss: 0.9681 - accuracy: 0.4800 - val_loss: 0.9275 - val_accuracy: 0.5400
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.9413 - accuracy: 0.50100/100 [==============================] - 0s 910us/sample - loss: 0.9014 - accuracy: 0.4600 - val_loss: 0.8519 - val_accuracy: 0.6600
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.9045 - accuracy: 0.56100/100 [==============================] - 0s 1ms/sample - loss: 0.8379 - accuracy: 0.7700 - val_loss: 0.7861 - val_accuracy: 1.0000
Epoch 5/5
 32/100 [========>.....................] 

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.2361 - accuracy: 0.40100/100 [==============================] - 0s 4ms/sample - loss: 1.0029 - accuracy: 0.5900 - val_loss: 0.5997 - val_accuracy: 0.9600
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.6390 - accuracy: 0.93100/100 [==============================] - 0s 977us/sample - loss: 0.6283 - accuracy: 0.7300 - val_loss: 0.4423 - val_accuracy: 1.0000
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.4679 - accuracy: 1.00100/100 [==============================] - 0s 180us/sample - loss: 0.4694 - accuracy: 0.9000 - val_loss: 0.3905 - val_accuracy: 0.8000
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.4635 - accuracy: 0.71100/100 [==============================] - 0s 180us/sample - loss: 0.4146 - accuracy: 0.7800 - val_loss: 0.2980 - val_accuracy: 0.9000
Epoch 5/5
 32/100 [========>.....................

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.2214 - accuracy: 0.31100/100 [==============================] - 0s 4ms/sample - loss: 1.1627 - accuracy: 0.3500 - val_loss: 1.1448 - val_accuracy: 0.3000
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.1442 - accuracy: 0.34100/100 [==============================] - 0s 189us/sample - loss: 1.1256 - accuracy: 0.3500 - val_loss: 1.1127 - val_accuracy: 0.3000
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.1492 - accuracy: 0.25100/100 [==============================] - 0s 189us/sample - loss: 1.0971 - accuracy: 0.3500 - val_loss: 1.0856 - val_accuracy: 0.3000
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0613 - accuracy: 0.37100/100 [==============================] - 0s 180us/sample - loss: 1.0698 - accuracy: 0.3500 - val_loss: 1.0621 - val_accuracy: 0.3000
Epoch 5/5
 32/100 [========>.....................

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.2342 - accuracy: 0.0000e+100/100 [==============================] - 0s 4ms/sample - loss: 1.1901 - accuracy: 0.0500 - val_loss: 1.1290 - val_accuracy: 0.3200
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.1202 - accuracy: 0.34100/100 [==============================] - 0s 180us/sample - loss: 1.0928 - accuracy: 0.3400 - val_loss: 1.0604 - val_accuracy: 0.3200
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0367 - accuracy: 0.34100/100 [==============================] - 0s 176us/sample - loss: 1.0284 - accuracy: 0.3400 - val_loss: 1.0053 - val_accuracy: 0.3200
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0178 - accuracy: 0.28100/100 [==============================] - 0s 906us/sample - loss: 0.9792 - accuracy: 0.3400 - val_loss: 0.9504 - val_accuracy: 0.3400
Epoch 5/5
 32/100 [========>.................

INFO:tensorflow:Oracle triggered exit


In [8]:
random_search_tuner.results_summary()


In [9]:
bayesian_opt_tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test)
             #validation_split=0.2,verbose=1)
             )

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0333 - accuracy: 0.28100/100 [==============================] - 0s 4ms/sample - loss: 1.0639 - accuracy: 0.3300 - val_loss: 1.3232 - val_accuracy: 0.3200
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.3889 - accuracy: 0.31100/100 [==============================] - 0s 918us/sample - loss: 1.1169 - accuracy: 0.4800 - val_loss: 0.5524 - val_accuracy: 0.6800
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.7179 - accuracy: 0.53100/100 [==============================] - 0s 918us/sample - loss: 0.5814 - accuracy: 0.7100 - val_loss: 0.4220 - val_accuracy: 0.7800
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.3992 - accuracy: 0.81100/100 [==============================] - 0s 953us/sample - loss: 0.4776 - accuracy: 0.6700 - val_loss: 0.3719 - val_accuracy: 0.8400
Epoch 5/5
 32/100 [========>.....................

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.4379 - accuracy: 0.25100/100 [==============================] - 0s 4ms/sample - loss: 1.2614 - accuracy: 0.3500 - val_loss: 1.1844 - val_accuracy: 0.3000
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0716 - accuracy: 0.40100/100 [==============================] - 0s 967us/sample - loss: 1.0703 - accuracy: 0.4000 - val_loss: 1.0554 - val_accuracy: 0.3200
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.1345 - accuracy: 0.28100/100 [==============================] - 0s 209us/sample - loss: 1.0139 - accuracy: 0.3400 - val_loss: 0.9939 - val_accuracy: 0.3200
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.9741 - accuracy: 0.34100/100 [==============================] - 0s 189us/sample - loss: 0.9657 - accuracy: 0.3400 - val_loss: 0.9338 - val_accuracy: 0.3200
Epoch 5/5
 32/100 [========>.....................

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0714 - accuracy: 0.34100/100 [==============================] - 0s 5ms/sample - loss: 1.0656 - accuracy: 0.3400 - val_loss: 1.0532 - val_accuracy: 0.3200
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0603 - accuracy: 0.31100/100 [==============================] - 0s 977us/sample - loss: 1.0478 - accuracy: 0.3400 - val_loss: 1.0403 - val_accuracy: 0.3400
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0634 - accuracy: 0.15100/100 [==============================] - 0s 1ms/sample - loss: 1.0339 - accuracy: 0.3700 - val_loss: 1.0283 - val_accuracy: 0.4400
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0175 - accuracy: 0.43100/100 [==============================] - 0s 1ms/sample - loss: 1.0203 - accuracy: 0.4300 - val_loss: 1.0161 - val_accuracy: 0.5200
Epoch 5/5
 32/100 [========>.....................] - 

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 3.0992 - accuracy: 0.28100/100 [==============================] - 0s 4ms/sample - loss: 2.4209 - accuracy: 0.3000 - val_loss: 1.3854 - val_accuracy: 0.3200
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.2903 - accuracy: 0.46100/100 [==============================] - 0s 180us/sample - loss: 1.3110 - accuracy: 0.3400 - val_loss: 1.1153 - val_accuracy: 0.3200
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.1134 - accuracy: 0.31100/100 [==============================] - 0s 928us/sample - loss: 1.0393 - accuracy: 0.3700 - val_loss: 1.0484 - val_accuracy: 0.4800
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.8468 - accuracy: 0.59100/100 [==============================] - 0s 947us/sample - loss: 0.9539 - accuracy: 0.5200 - val_loss: 0.8879 - val_accuracy: 0.5200
Epoch 5/5
 32/100 [========>.....................

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0515 - accuracy: 0.71100/100 [==============================] - 0s 4ms/sample - loss: 0.9073 - accuracy: 0.6200 - val_loss: 0.6350 - val_accuracy: 0.6600
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.8147 - accuracy: 0.53100/100 [==============================] - 0s 947us/sample - loss: 0.5960 - accuracy: 0.6400 - val_loss: 0.4080 - val_accuracy: 0.8800
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.4616 - accuracy: 0.87100/100 [==============================] - 0s 938us/sample - loss: 0.4368 - accuracy: 0.8700 - val_loss: 0.3227 - val_accuracy: 0.9000
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.3433 - accuracy: 0.96100/100 [==============================] - 0s 928us/sample - loss: 0.3910 - accuracy: 0.8900 - val_loss: 0.2579 - val_accuracy: 0.9800
Epoch 5/5
 32/100 [========>.....................

INFO:tensorflow:Oracle triggered exit


In [10]:
bayesian_opt_tuner.results_summary()


## Best models achieved with the random search and and bayesian hyperparametrization

In [11]:
rand_searched_models = random_search_tuner.get_best_models(num_models=-1)
bayes_optimized_models = bayesian_opt_tuner.get_best_models(num_models=-1)

#tuner.get_best_hyperparameters()
print('number of random searched models: {}'.format(len(rand_searched_models)))
print('number of bayesian optimized models: {}'.format(len(bayes_optimized_models)))


number of random searched models: 4
number of bayesian optimized models: 4


## Get model weights:

In [12]:
random_searched_model_best_model = random_search_tuner.get_best_models(num_models=1)
bayes_opt_model_best_model = bayesian_opt_tuner.get_best_models(num_models=1)



 ### Evaluation score:
print('single prediction on a test instance: {}'.format(models[0].predict(X_test[-1].reshape(1, -1))))

In [13]:
print('single prediction of the random_searched_model_best_model on a test instance: {}'.format(random_searched_model_best_model[0].predict(X_test[-1].reshape(1, -1))))
print('single prediction of the bayes_opt_model_best_model on a test instance: {}'.format(bayes_opt_model_best_model[0].predict(X_test[-1].reshape(1, -1))))


single prediction of the random_searched_model_best_model on a test instance: [[0.08193266 0.42444137 0.49362594]]
single prediction of the bayes_opt_model_best_model on a test instance: [[3.1111806e-04 1.7690873e-01 8.2278013e-01]]


## Predictions on the test set:

In [14]:
test_set_random_searched_best_model_predictions = random_searched_model_best_model[0].predict(X_test)
test_set_bayes_opt_best_model_predictions = bayes_opt_model_best_model[0].predict(X_test)


### Esperamos obtener un score = 0.90 para el best_model_0 según el 'Results summary'

In [15]:
import tensorflow

categ_acc_rand_search_test_set = tensorflow.keras.metrics.CategoricalAccuracy()
#best_model_0_eval_set_acc = tensorflow.keras.metrics.CategoricalAccuracy(y_test.reshape(-1,3), test_set_predictions.reshape(-1,3))
_ = categ_acc_rand_search_test_set.update_state(y_test, test_set_random_searched_best_model_predictions)
print('random_searched_model_best_model_eval_set_acc: {}'.format(categ_acc_rand_search_test_set.result().numpy()))

categ_acc_bayes_opt_test_set = tensorflow.keras.metrics.CategoricalAccuracy()
_ = categ_acc_bayes_opt_test_set.update_state(y_test, test_set_bayes_opt_best_model_predictions)
print('bayes_opt_model_best_model_eval_set_acc: {}'.format(categ_acc_bayes_opt_test_set.result().numpy()))


random_searched_model_best_model_eval_set_acc: 1.0
bayes_opt_model_best_model_eval_set_acc: 0.9800000190734863


## Re-run model trainings and hyperparam. pero con diferente número de max_trials (esto es, hyperparams combinations)

In [16]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

### Testing keras tuner on IRIS dataset

In [17]:
from sklearn.datasets import load_iris
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

iris_ds = load_iris()
X = iris_ds.data
# convert integers to dummy variables (i.e. one hot encoded)
y_one_hot_encoded = to_categorical(iris_ds.target)
y_one_hot_encoded[:4]


array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot_encoded, test_size=0.33, random_state=42)



Usage: the basics
Here's how to perform hyperparameter tuning for a single-layer dense neural network using: 
  * random search
  * bayesian optimization
  * hyperband

First, we define a model-building function. It takes an argument hp from which you can sample hyperparameters, such as hp.Int('units', min_value=32, max_value=512, step=32) (an integer from a certain range).

This function returns a compiled model.

In [19]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(input_dim=X.shape[1],
                            units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                            
                           activation='relu'))
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model


Next, instantiate a tuner. You should specify the model-building function, the name of the objective to optimize (whether to minimize or maximize is automatically inferred for built-in metrics), the total number of trials (max_trials) to test, and the number of models that should be built and fit for each trial (executions_per_trial).

Available tuners are RandomSearch and Hyperband (y bayesian?)

Note: the purpose of having multiple executions per trial is to reduce results variance and therefore be able to more accurately assess the performance of a model. If you want to get results faster, you could set executions_per_trial=1 (single round of training for each model configuration).

In [20]:
import os

random_search_tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_random_search_tuner_test_2',
    overwrite=True)


In [21]:
from kerastuner import BayesianOptimization

bayesian_opt_tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=1,
    executions_per_trial=1,
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_bayesian_opt_tuner_test_2',
    overwrite=True)


summary of the search space:
Start the search for the best hyperparameter configuration. The call to search has the same signature as model.fit().
Here's what happens in search: models are built iteratively by calling the model-building function, which populates the hyperparameter space (search space) tracked by the hp object. The tuner progressively explores the space, recording metrics for each configuration.

In [22]:
random_search_tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test)
             #validation_split=0.2,verbose=1)
             )


Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.2436 - accuracy: 0.37100/100 [==============================] - 0s 4ms/sample - loss: 1.2129 - accuracy: 0.3400 - val_loss: 1.1148 - val_accuracy: 0.3200
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.1118 - accuracy: 0.25100/100 [==============================] - 0s 209us/sample - loss: 1.0736 - accuracy: 0.3400 - val_loss: 1.0451 - val_accuracy: 0.3200
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0477 - accuracy: 0.31100/100 [==============================] - 0s 816us/sample - loss: 1.0087 - accuracy: 0.3400 - val_loss: 0.9411 - val_accuracy: 0.3400
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0139 - accuracy: 0.25100/100 [==============================] - 0s 808us/sample - loss: 0.9235 - accuracy: 0.5000 - val_loss: 0.8475 - val_accuracy: 0.7000
Epoch 5/5
 32/100 [========>.....................

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0708 - accuracy: 0.37100/100 [==============================] - 0s 4ms/sample - loss: 1.0759 - accuracy: 0.3500 - val_loss: 1.0748 - val_accuracy: 0.3000
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0515 - accuracy: 0.37100/100 [==============================] - 0s 199us/sample - loss: 1.0529 - accuracy: 0.3500 - val_loss: 1.0515 - val_accuracy: 0.3000
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0557 - accuracy: 0.28100/100 [==============================] - 0s 199us/sample - loss: 1.0350 - accuracy: 0.3500 - val_loss: 1.0322 - val_accuracy: 0.3000
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0167 - accuracy: 0.34100/100 [==============================] - 0s 783us/sample - loss: 1.0180 - accuracy: 0.3700 - val_loss: 1.0174 - val_accuracy: 0.4600
Epoch 5/5
 32/100 [========>.....................

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.2735 - accuracy: 0.21100/100 [==============================] - 0s 4ms/sample - loss: 1.2388 - accuracy: 0.2700 - val_loss: 1.0913 - val_accuracy: 0.4800
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0578 - accuracy: 0.50100/100 [==============================] - 0s 807us/sample - loss: 0.9213 - accuracy: 0.5900 - val_loss: 0.6974 - val_accuracy: 0.7000
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.8826 - accuracy: 0.59100/100 [==============================] - 0s 798us/sample - loss: 0.7277 - accuracy: 0.6500 - val_loss: 0.4899 - val_accuracy: 0.9200
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.5222 - accuracy: 0.96100/100 [==============================] - 0s 199us/sample - loss: 0.5655 - accuracy: 0.7700 - val_loss: 0.5053 - val_accuracy: 0.6800
Epoch 5/5
 32/100 [========>.....................

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0532 - accuracy: 0.31100/100 [==============================] - 0s 4ms/sample - loss: 1.0413 - accuracy: 0.4800 - val_loss: 0.7923 - val_accuracy: 0.5000
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.8257 - accuracy: 0.43100/100 [==============================] - 0s 708us/sample - loss: 0.8347 - accuracy: 0.4000 - val_loss: 0.7403 - val_accuracy: 0.5800
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.7255 - accuracy: 0.53100/100 [==============================] - 0s 748us/sample - loss: 0.6745 - accuracy: 0.7400 - val_loss: 0.5836 - val_accuracy: 0.7000
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.7289 - accuracy: 0.56100/100 [==============================] - 0s 189us/sample - loss: 0.6296 - accuracy: 0.6500 - val_loss: 0.5105 - val_accuracy: 0.7000
Epoch 5/5
 32/100 [========>.....................

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.9626 - accuracy: 0.59100/100 [==============================] - 0s 4ms/sample - loss: 1.0057 - accuracy: 0.3800 - val_loss: 0.9222 - val_accuracy: 0.3200
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.9250 - accuracy: 0.31100/100 [==============================] - 0s 818us/sample - loss: 0.8975 - accuracy: 0.3500 - val_loss: 0.8417 - val_accuracy: 0.7800
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.8345 - accuracy: 0.68100/100 [==============================] - 0s 768us/sample - loss: 0.8318 - accuracy: 0.7900 - val_loss: 0.7777 - val_accuracy: 0.9800
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.7964 - accuracy: 0.90100/100 [==============================] - 0s 189us/sample - loss: 0.7817 - accuracy: 0.8800 - val_loss: 0.7178 - val_accuracy: 0.8600
Epoch 5/5
 32/100 [========>.....................

INFO:tensorflow:Oracle triggered exit


In [23]:
random_search_tuner.results_summary()


In [24]:
bayesian_opt_tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test)
             #validation_split=0.2,verbose=1)
             )

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.1159 - accuracy: 0.43100/100 [==============================] - 0s 4ms/sample - loss: 1.0793 - accuracy: 0.3400 - val_loss: 0.9589 - val_accuracy: 0.3200
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.9677 - accuracy: 0.25100/100 [==============================] - 0s 798us/sample - loss: 0.9173 - accuracy: 0.5700 - val_loss: 0.8378 - val_accuracy: 0.7000
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.8429 - accuracy: 0.71100/100 [==============================] - 0s 783us/sample - loss: 0.8401 - accuracy: 0.6500 - val_loss: 0.7593 - val_accuracy: 0.7200
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.8112 - accuracy: 0.62100/100 [==============================] - 0s 808us/sample - loss: 0.7739 - accuracy: 0.8400 - val_loss: 0.7037 - val_accuracy: 0.9200
Epoch 5/5
 32/100 [========>.....................

INFO:tensorflow:Oracle triggered exit


In [25]:
bayesian_opt_tuner.results_summary()


## Best models achieved with the random search and and bayesian hyperparametrization

In [26]:
rand_searched_models = random_search_tuner.get_best_models(num_models=-1)
bayes_optimized_models = bayesian_opt_tuner.get_best_models(num_models=-1)

#tuner.get_best_hyperparameters()
print('number of random searched models: {}'.format(len(rand_searched_models)))
print('number of bayesian optimized models: {}'.format(len(bayes_optimized_models)))


number of random searched models: 4
number of bayesian optimized models: 0


## Get model weights:

In [27]:
random_searched_model_best_model = random_search_tuner.get_best_models(num_models=1)
bayes_opt_model_best_model = bayesian_opt_tuner.get_best_models(num_models=1)



 ### Evaluation score:
print('single prediction on a test instance: {}'.format(models[0].predict(X_test[-1].reshape(1, -1))))

In [28]:
print('single prediction of the random_searched_model_best_model on a test instance: {}'.format(random_searched_model_best_model[0].predict(X_test[-1].reshape(1, -1))))
print('single prediction of the bayes_opt_model_best_model on a test instance: {}'.format(bayes_opt_model_best_model[0].predict(X_test[-1].reshape(1, -1))))


single prediction of the random_searched_model_best_model on a test instance: [[0.00113969 0.35658044 0.64227986]]
single prediction of the bayes_opt_model_best_model on a test instance: [[0.07741494 0.41732666 0.5052584 ]]


## Predictions on the test set:

In [29]:
test_set_random_searched_best_model_predictions = random_searched_model_best_model[0].predict(X_test)
test_set_bayes_opt_best_model_predictions = bayes_opt_model_best_model[0].predict(X_test)


### Esperamos obtener un score = 0.90 para el best_model_0 según el 'Results summary'

In [30]:
import tensorflow

categ_acc_rand_search_test_set = tensorflow.keras.metrics.CategoricalAccuracy()
#best_model_0_eval_set_acc = tensorflow.keras.metrics.CategoricalAccuracy(y_test.reshape(-1,3), test_set_predictions.reshape(-1,3))
_ = categ_acc_rand_search_test_set.update_state(y_test, test_set_random_searched_best_model_predictions)
print('random_searched_model_best_model_eval_set_acc: {}'.format(categ_acc_rand_search_test_set.result().numpy()))

categ_acc_bayes_opt_test_set = tensorflow.keras.metrics.CategoricalAccuracy()
_ = categ_acc_bayes_opt_test_set.update_state(y_test, test_set_bayes_opt_best_model_predictions)
print('bayes_opt_model_best_model_eval_set_acc: {}'.format(categ_acc_bayes_opt_test_set.result().numpy()))


random_searched_model_best_model_eval_set_acc: 0.9800000190734863
bayes_opt_model_best_model_eval_set_acc: 0.9800000190734863


## Re-run model trainings and hyperparam. pero con diferente número de max_trials (esto es, hyperparams combinations)

In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

### Testing keras tuner on IRIS dataset

In [2]:
from sklearn.datasets import load_iris
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

iris_ds = load_iris()
X = iris_ds.data
# convert integers to dummy variables (i.e. one hot encoded)
y_one_hot_encoded = to_categorical(iris_ds.target)
y_one_hot_encoded[:4]


array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot_encoded, test_size=0.33, random_state=42)



Usage: the basics
Here's how to perform hyperparameter tuning for a single-layer dense neural network using: 
  * random search
  * bayesian optimization
  * hyperband

First, we define a model-building function. It takes an argument hp from which you can sample hyperparameters, such as hp.Int('units', min_value=32, max_value=512, step=32) (an integer from a certain range).

This function returns a compiled model.

In [4]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(input_dim=X.shape[1],
                            units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                            
                           activation='relu'))
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model


Next, instantiate a tuner. You should specify the model-building function, the name of the objective to optimize (whether to minimize or maximize is automatically inferred for built-in metrics), the total number of trials (max_trials) to test, and the number of models that should be built and fit for each trial (executions_per_trial).

Available tuners are RandomSearch and Hyperband (y bayesian?)

Note: the purpose of having multiple executions per trial is to reduce results variance and therefore be able to more accurately assess the performance of a model. If you want to get results faster, you could set executions_per_trial=1 (single round of training for each model configuration).

In [5]:
import os

random_search_tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=1,
    executions_per_trial=1,
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_random_search_tuner_test_2',
    overwrite=True)


In [6]:
from kerastuner import BayesianOptimization

bayesian_opt_tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=1,
    executions_per_trial=1,
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_bayesian_opt_tuner_test_2',
    overwrite=True)


summary of the search space:
Start the search for the best hyperparameter configuration. The call to search has the same signature as model.fit().
Here's what happens in search: models are built iteratively by calling the model-building function, which populates the hyperparameter space (search space) tracked by the hp object. The tuner progressively explores the space, recording metrics for each configuration.

In [7]:
random_search_tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test)
             #validation_split=0.2,verbose=1)
             )


Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 1s - loss: 1.0394 - accuracy: 0.65100/100 [==============================] - 1s 8ms/sample - loss: 1.0321 - accuracy: 0.4100 - val_loss: 0.9536 - val_accuracy: 0.5600
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.9534 - accuracy: 0.59100/100 [==============================] - 0s 983us/sample - loss: 0.9305 - accuracy: 0.6200 - val_loss: 0.8507 - val_accuracy: 0.7000
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.8466 - accuracy: 0.75100/100 [==============================] - 0s 199us/sample - loss: 0.8618 - accuracy: 0.6500 - val_loss: 0.7816 - val_accuracy: 0.7000
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.8444 - accuracy: 0.59100/100 [==============================] - 0s 189us/sample - loss: 0.8023 - accuracy: 0.6500 - val_loss: 0.7379 - val_accuracy: 0.7000
Epoch 5/5
 32/100 [========>.....................

INFO:tensorflow:Oracle triggered exit


In [8]:
random_search_tuner.results_summary()


In [9]:
bayesian_opt_tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test)
             #validation_split=0.2,verbose=1)
             )

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.1780 - accuracy: 0.40100/100 [==============================] - 0s 4ms/sample - loss: 1.1210 - accuracy: 0.5100 - val_loss: 0.9449 - val_accuracy: 0.6800
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.9724 - accuracy: 0.65100/100 [==============================] - 0s 194us/sample - loss: 0.9235 - accuracy: 0.4700 - val_loss: 0.9066 - val_accuracy: 0.4200
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.9118 - accuracy: 0.40100/100 [==============================] - 0s 199us/sample - loss: 0.8723 - accuracy: 0.4300 - val_loss: 0.8182 - val_accuracy: 0.4400
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.8013 - accuracy: 0.43100/100 [==============================] - 0s 1ms/sample - loss: 0.7959 - accuracy: 0.5500 - val_loss: 0.7184 - val_accuracy: 0.7000
Epoch 5/5
 32/100 [========>.....................] 

INFO:tensorflow:Oracle triggered exit


In [10]:
bayesian_opt_tuner.results_summary()


## Best models achieved with the random search and and bayesian hyperparametrization

In [11]:
rand_searched_models = random_search_tuner.get_best_models(num_models=-1)
bayes_optimized_models = bayesian_opt_tuner.get_best_models(num_models=-1)

#tuner.get_best_hyperparameters()
print('number of random searched models: {}'.format(len(rand_searched_models)))
print('number of bayesian optimized models: {}'.format(len(bayes_optimized_models)))


number of random searched models: 0
number of bayesian optimized models: 0


## Get model weights:

In [12]:
random_searched_model_best_model = random_search_tuner.get_best_models(num_models=1)
bayes_opt_model_best_model = bayesian_opt_tuner.get_best_models(num_models=1)



 ### Evaluation score:
print('single prediction on a test instance: {}'.format(models[0].predict(X_test[-1].reshape(1, -1))))

In [13]:
print('single prediction of the random_searched_model_best_model on a test instance: {}'.format(random_searched_model_best_model[0].predict(X_test[-1].reshape(1, -1))))
print('single prediction of the bayes_opt_model_best_model on a test instance: {}'.format(bayes_opt_model_best_model[0].predict(X_test[-1].reshape(1, -1))))


single prediction of the random_searched_model_best_model on a test instance: [[0.15195647 0.3824394  0.4656042 ]]
single prediction of the bayes_opt_model_best_model on a test instance: [[0.08514515 0.36450717 0.5503476 ]]


## Predictions on the test set:

In [14]:
test_set_random_searched_best_model_predictions = random_searched_model_best_model[0].predict(X_test)
test_set_bayes_opt_best_model_predictions = bayes_opt_model_best_model[0].predict(X_test)


### Esperamos obtener un score = 0.90 para el best_model_0 según el 'Results summary'

In [15]:
import tensorflow

categ_acc_rand_search_test_set = tensorflow.keras.metrics.CategoricalAccuracy()
#best_model_0_eval_set_acc = tensorflow.keras.metrics.CategoricalAccuracy(y_test.reshape(-1,3), test_set_predictions.reshape(-1,3))
_ = categ_acc_rand_search_test_set.update_state(y_test, test_set_random_searched_best_model_predictions)
print('random_searched_model_best_model_eval_set_acc: {}'.format(categ_acc_rand_search_test_set.result().numpy()))

categ_acc_bayes_opt_test_set = tensorflow.keras.metrics.CategoricalAccuracy()
_ = categ_acc_bayes_opt_test_set.update_state(y_test, test_set_bayes_opt_best_model_predictions)
print('bayes_opt_model_best_model_eval_set_acc: {}'.format(categ_acc_bayes_opt_test_set.result().numpy()))


random_searched_model_best_model_eval_set_acc: 0.699999988079071
bayes_opt_model_best_model_eval_set_acc: 0.9200000166893005


In [1]:
### Al lanzar con un solo max_trial, vemos que coincide nuestro 'CategoricalAccuracy' con el 'score' informado en el summary

In [2]:
# Save the best models, try also to save it with the built-in option:

from tensorflow.keras.models import model_from_json

directory=os.path.normpath('C:/keras_tuning/iris_data_keras_bayesian_opt_tuner_test_2')
# serialize model to JSON
bayesian_model_json = bayes_opt_model_best_model[0].to_json()
with open("model.json", "w") as json_file:
    json_file.write(bayesian_model_json)
# serialize weights to HDF5
bayes_opt_model_best_model[0].save_weights("model.h5")
print("Saved model to disk")




NameError: name 'bayes_opt_model_best_model' is not defined

In [3]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

### Testing keras tuner on IRIS dataset

In [4]:
from sklearn.datasets import load_iris
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

iris_ds = load_iris()
X = iris_ds.data
# convert integers to dummy variables (i.e. one hot encoded)
y_one_hot_encoded = to_categorical(iris_ds.target)
y_one_hot_encoded[:4]


array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot_encoded, test_size=0.33, random_state=42)



Usage: the basics
Here's how to perform hyperparameter tuning for a single-layer dense neural network using: 
  * random search
  * bayesian optimization
  * hyperband

First, we define a model-building function. It takes an argument hp from which you can sample hyperparameters, such as hp.Int('units', min_value=32, max_value=512, step=32) (an integer from a certain range).

This function returns a compiled model.

In [6]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(input_dim=X.shape[1],
                            units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                            
                           activation='relu'))
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model


Next, instantiate a tuner. You should specify the model-building function, the name of the objective to optimize (whether to minimize or maximize is automatically inferred for built-in metrics), the total number of trials (max_trials) to test, and the number of models that should be built and fit for each trial (executions_per_trial).

Available tuners are RandomSearch and Hyperband (y bayesian?)

Note: the purpose of having multiple executions per trial is to reduce results variance and therefore be able to more accurately assess the performance of a model. If you want to get results faster, you could set executions_per_trial=1 (single round of training for each model configuration).

In [7]:
import os

random_search_tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=1,
    executions_per_trial=1,
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_random_search_tuner_test_2',
    overwrite=True)


In [8]:
from kerastuner import BayesianOptimization

bayesian_opt_tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=1,
    executions_per_trial=1,
    directory=os.path.normpath('C:/keras_tuning'),
    project_name='iris_data_keras_bayesian_opt_tuner_test_2',
    overwrite=True)


summary of the search space:
Start the search for the best hyperparameter configuration. The call to search has the same signature as model.fit().
Here's what happens in search: models are built iteratively by calling the model-building function, which populates the hyperparameter space (search space) tracked by the hp object. The tuner progressively explores the space, recording metrics for each configuration.

In [9]:
random_search_tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test)
             #validation_split=0.2,verbose=1)
             )


Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 1s - loss: 1.1483 - accuracy: 0.0000e+100/100 [==============================] - 1s 8ms/sample - loss: 1.0586 - accuracy: 0.3100 - val_loss: 0.7178 - val_accuracy: 0.6600
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.7164 - accuracy: 0.71100/100 [==============================] - 0s 768us/sample - loss: 0.7975 - accuracy: 0.6300 - val_loss: 0.5403 - val_accuracy: 0.7000
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.5618 - accuracy: 0.68100/100 [==============================] - 0s 180us/sample - loss: 0.7734 - accuracy: 0.6500 - val_loss: 0.5402 - val_accuracy: 0.7000
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 0.7486 - accuracy: 0.46100/100 [==============================] - 0s 170us/sample - loss: 0.6169 - accuracy: 0.6100 - val_loss: 0.4960 - val_accuracy: 0.6800
Epoch 5/5
 32/100 [========>.................

INFO:tensorflow:Oracle triggered exit


In [10]:
random_search_tuner.results_summary()


In [11]:
bayesian_opt_tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test)
             #validation_split=0.2,verbose=1)
             )

Train on 100 samples, validate on 50 samples
Epoch 1/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.3701 - accuracy: 0.25100/100 [==============================] - 0s 4ms/sample - loss: 1.2249 - accuracy: 0.3400 - val_loss: 1.2083 - val_accuracy: 0.3400
Epoch 2/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.2240 - accuracy: 0.31100/100 [==============================] - 0s 763us/sample - loss: 1.1488 - accuracy: 0.4600 - val_loss: 1.1662 - val_accuracy: 0.5000
Epoch 3/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.2085 - accuracy: 0.37100/100 [==============================] - 0s 189us/sample - loss: 1.0977 - accuracy: 0.5300 - val_loss: 1.0948 - val_accuracy: 0.4800
Epoch 4/5
 32/100 [========>.....................] - ETA: 0s - loss: 1.0650 - accuracy: 0.56100/100 [==============================] - 0s 249us/sample - loss: 1.0422 - accuracy: 0.5300 - val_loss: 1.0411 - val_accuracy: 0.4800
Epoch 5/5
 32/100 [========>.....................

INFO:tensorflow:Oracle triggered exit


In [12]:
bayesian_opt_tuner.results_summary()


## Best models achieved with the random search and and bayesian hyperparametrization

In [13]:
rand_searched_models = random_search_tuner.get_best_models(num_models=-1)
bayes_optimized_models = bayesian_opt_tuner.get_best_models(num_models=-1)

#tuner.get_best_hyperparameters()
print('number of random searched models: {}'.format(len(rand_searched_models)))
print('number of bayesian optimized models: {}'.format(len(bayes_optimized_models)))


number of random searched models: 0
number of bayesian optimized models: 0


## Get model weights:

In [14]:
random_searched_model_best_model = random_search_tuner.get_best_models(num_models=1)
bayes_opt_model_best_model = bayesian_opt_tuner.get_best_models(num_models=1)


 ### Evaluation score:
print('single prediction on a test instance: {}'.format(models[0].predict(X_test[-1].reshape(1, -1))))

In [15]:
print('single prediction of the random_searched_model_best_model on a test instance: {}'.format(random_searched_model_best_model[0].predict(X_test[-1].reshape(1, -1))))
print('single prediction of the bayes_opt_model_best_model on a test instance: {}'.format(bayes_opt_model_best_model[0].predict(X_test[-1].reshape(1, -1))))


single prediction of the random_searched_model_best_model on a test instance: [[0.0168224  0.1594499  0.82372767]]
single prediction of the bayes_opt_model_best_model on a test instance: [[0.13170053 0.43555695 0.43274254]]


## Predictions on the test set:

In [16]:
test_set_random_searched_best_model_predictions = random_searched_model_best_model[0].predict(X_test)
test_set_bayes_opt_best_model_predictions = bayes_opt_model_best_model[0].predict(X_test)


### Esperamos obtener un score = 0.90 para el best_model_0 según el 'Results summary'

In [17]:
import tensorflow

categ_acc_rand_search_test_set = tensorflow.keras.metrics.CategoricalAccuracy()
#best_model_0_eval_set_acc = tensorflow.keras.metrics.CategoricalAccuracy(y_test.reshape(-1,3), test_set_predictions.reshape(-1,3))
_ = categ_acc_rand_search_test_set.update_state(y_test, test_set_random_searched_best_model_predictions)
print('random_searched_model_best_model_eval_set_acc: {}'.format(categ_acc_rand_search_test_set.result().numpy()))

categ_acc_bayes_opt_test_set = tensorflow.keras.metrics.CategoricalAccuracy()
_ = categ_acc_bayes_opt_test_set.update_state(y_test, test_set_bayes_opt_best_model_predictions)
print('bayes_opt_model_best_model_eval_set_acc: {}'.format(categ_acc_bayes_opt_test_set.result().numpy()))


random_searched_model_best_model_eval_set_acc: 0.699999988079071
bayes_opt_model_best_model_eval_set_acc: 0.5199999809265137


In [18]:
# Save the best models, try also to save it with the built-in option:

from tensorflow.keras.models import model_from_json

directory=os.path.normpath('C:/keras_tuning/iris_data_keras_bayesian_opt_tuner_test_2')
# serialize model to JSON
bayesian_model_json = bayes_opt_model_best_model[0].to_json()
with open("model.json", "w") as json_file:
    json_file.write(bayesian_model_json)
# serialize weights to HDF5
bayes_opt_model_best_model[0].save_weights("model.h5")
print("Saved model to disk")



Saved model to disk


In [19]:
# Save the best models, try also to save it with the built-in option:

from tensorflow.keras.models import model_from_json

directory_model_arch_json=os.path.normpath('C:/keras_tuning/iris_data_keras_bayesian_opt_tuner_test_2/model.json')
directory_model_h5=os.path.normpath('C:/keras_tuning/iris_data_keras_bayesian_opt_tuner_test_2/model.h5')
# serialize model to JSON
bayesian_model_json = bayes_opt_model_best_model[0].to_json()
with open(directory_model_arch_json, "w") as json_file:
    json_file.write(bayesian_model_json)
# serialize weights to HDF5
bayes_opt_model_best_model[0].save_weights(directory_model_h5)
print("Saved model to disk")



Saved model to disk


In [21]:
# Load model:
from tensorflow.keras.models import model_from_json
# load json and create model
json_file = open(directory_model_arch_json, 'r')
loaded_model_json = json_file.read()
json_file.close()

loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(directory_model_h5)

In [22]:
print("Loaded model from disk")
# evaluate loaded model on test data
'''
#loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))
'''
loaded_model.predict(X_test)



Loaded model from disk


array([[0.18691356, 0.40708265, 0.4060038 ],
       [0.28102165, 0.500248  , 0.21873033],
       [0.12444609, 0.3484776 , 0.5270763 ],
       [0.1869745 , 0.43074673, 0.38227883],
       [0.19077368, 0.4244052 , 0.3848211 ],
       [0.29011714, 0.49235243, 0.21753041],
       [0.22125033, 0.45003098, 0.32871872],
       [0.16334489, 0.46057817, 0.37607694],
       [0.19597395, 0.40819627, 0.3958298 ],
       [0.2158887 , 0.43205994, 0.35205135],
       [0.16219898, 0.44247136, 0.39532968],
       [0.3044084 , 0.45945838, 0.23613323],
       [0.29613838, 0.49743438, 0.20642725],
       [0.30076176, 0.4607826 , 0.23845556],
       [0.27742857, 0.49995244, 0.222619  ],
       [0.17926702, 0.44602126, 0.37471175],
       [0.14081255, 0.39579558, 0.46339187],
       [0.21615186, 0.4175978 , 0.36625034],
       [0.18703076, 0.41160712, 0.40136215],
       [0.14627685, 0.39922458, 0.4544986 ],
       [0.2903147 , 0.46196306, 0.24772227],
       [0.16836071, 0.42768717, 0.40395215],
       [0.

In [23]:
print("Loaded model from disk")
# evaluate loaded model on test data
'''
#loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))
'''
#loaded_model.predict(X_test)
test_set_bayes_opt_loaded_model_predictions = loaded_model.predict(X_test)
categ_acc_bayes_opt_loaded_model_test_set = tensorflow.keras.metrics.CategoricalAccuracy()
_ = categ_acc_bayes_opt_loaded_model_test_set.update_state(y_test, test_set_bayes_opt_loaded_model_predictions)
print('bayes_opt_model_best_model_eval_set_acc: {}'.format(categ_acc_bayes_opt_loaded_model_test_set.result().numpy()))



Loaded model from disk
bayes_opt_model_best_model_eval_set_acc: 0.5199999809265137
